In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as A
import albumentations.pytorch
from tqdm.notebook import tqdm

import cv2
import re
import time

import sys
sys.path.append('../utils')
sys.path.append('../')
from visualize import *
from xml_parser import *
from transformations import *
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler

In [ ]:
# src_path = '/home/Dataset/Papsmear/original'
# df = pd.DataFrame(columns=['label', 'org_label', 'bbox', 'xmin','ymin','xmax','ymax', 'xmlh', 'xmlw', 'file_name', 'path'])
# # df = pd.DataFrame(columns=['class','bbox', 'path'])
# data_list = []
# for dir_name in ['SS', 'SS2']:
#     dpath = os.path.join(src_path, dir_name)
# #     print(dpath)
#     dir_lists = os.listdir(dpath) 
# #     print(dir_lists)
    
#     for dir_list in dir_lists :
#         dpath2 = os.path.join(dpath, dir_list)
#         xml_list = [os.path.join(dpath2, d) for d in os.listdir(dpath2) if d.endswith(".xml")]
# #         print(xml_list)
#         for xml_path in xml_list :
#             parser = XMLParser(xml_path)
#             if (parser.height, parser.width) not in parser.rejection_size:  
#                 filename = parser.file_name
#                 for obj in parser.objects :
#                     obj_class = obj[0]
#                     if obj_class in rej_table :
#                         continue
                    
#                     img_path = '/'.join((xml_path.split('/')[:-1])) + '/' + filename
#     #                 img_path = re.sub('xml', 'jpg', xml_path)
#     # #                     print(img_path)
#                     obj.append(parser.height)
#                     obj.append(parser.width)
#                     obj.append(parser.file_name)
#                     obj.append(img_path)
#                     data_list.append(obj)
#     # #                     print(obj)

In [2]:
df = pd.read_csv('df.csv')
df['xdiff'] = df.apply(lambda x : x['xmax'] - x['xmin'], axis=1)
df['ydiff'] = df.apply(lambda x : x['ymax'] - x['ymin'], axis=1)
df['area'] = df.apply(lambda x : x['xdiff'] * x['ydiff'], axis=1)
df['format'] = df.file_name.apply(lambda x : x.split('.')[-1])
 
df = df[['file_name', 'xmlh', 'xmlw', 'res', 'h', 'w', 'xdiff', 'ydiff',
       'area', 'bbox', 'xmin', 'ymin', 'xmax', 'ymax',
       'label', 'org_label', 'format', 'path']]
df.head()

,file_name,xmlh,xmlw,res,h,w,xdiff,ydiff,area,bbox,xmin,ymin,xmax,ymax,label,org_label,format,path
0,20200623_100015.jpg,1960,4032,"(4032, 1960)",4032,1960,117,124,14508,"[2061, 1051, 2178, 1175]",2061,1051,2178,1175,Carcinoma,Suamous cell carcinoma,jpg,/home/Dataset/Papsmear/original/SS/06/20200623...
1,20200420_100401.jpg,1960,4032,"(4032, 1960)",4032,1960,79,71,5609,"[1945, 870, 2024, 941]",1945,870,2024,941,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS/06/20200420...
2,20200420_100401.jpg,1960,4032,"(4032, 1960)",4032,1960,84,73,6132,"[2400, 947, 2484, 1020]",2400,947,2484,1020,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS/06/20200420...
3,20200420_100401.jpg,1960,4032,"(4032, 1960)",4032,1960,98,67,6566,"[2209, 845, 2307, 912]",2209,845,2307,912,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS/06/20200420...
4,20200420_100401.jpg,1960,4032,"(4032, 1960)",4032,1960,88,55,4840,"[2056, 811, 2144, 866]",2056,811,2144,866,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS/06/20200420...


In [6]:
print(df.shape)
df = df[df['xdiff'] > 25]
df = df[df['ydiff'] > 25]
df = df[df['area'] > 1000]
print(df.shape)
df.reset_index(drop=True, inplace=True)
# df.head()

(100871, 18)
(100871, 18)


In [7]:
set(df.label)

{'ASCUS', 'Carcinoma', 'HSIL', 'LSIL', 'Normal'}

In [5]:
df.tail()

,file_name,xmlh,xmlw,res,h,w,xdiff,ydiff,area,bbox,xmin,ymin,xmax,ymax,label,org_label,format,path
100866,20200903_104408.jpg,1800,4000,"(4000, 1800)",4000,1800,98,138,13524,"[2165, 1052, 2263, 1190]",2165,1052,2263,1190,ASCUS,ASCUS,jpg,/home/Dataset/Papsmear/original/SS2/02(200901-...
100867,20200903_093016.jpg,1800,4000,"(4000, 1800)",4000,1800,160,122,19520,"[2208, 523, 2368, 645]",2208,523,2368,645,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS2/02(200901-...
100868,20200903_093016.jpg,1800,4000,"(4000, 1800)",4000,1800,193,91,17563,"[2091, 737, 2284, 828]",2091,737,2284,828,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS2/02(200901-...
100869,20200903_093016.jpg,1800,4000,"(4000, 1800)",4000,1800,139,116,16124,"[2124, 828, 2263, 944]",2124,828,2263,944,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS2/02(200901-...
100870,20200903_093016.jpg,1800,4000,"(4000, 1800)",4000,1800,137,107,14659,"[2480, 876, 2617, 983]",2480,876,2617,983,HSIL,HSIL,jpg,/home/Dataset/Papsmear/original/SS2/02(200901-...


In [8]:
encoder = LabelEncoder()

# X_train데이터를 이용 피팅하고 라벨숫자로 변환한다
encoder.fit(df.label)
df['label_id'] = encoder.transform(df.label)
df = df[['file_name', 'xmlh', 'xmlw', 'res', 'h', 'w', 'xdiff', 'ydiff', 'area',
       'bbox', 'xmin', 'ymin', 'xmax', 'ymax', 'label_id', 'label', 'org_label', 'format',
       'path']]
#df.tail()


In [9]:
df[df['xmlh'] != df['h']].h.unique()

array([4032, 4000])

In [10]:
image_list = df.path.unique()
type(image_list)

numpy.ndarray

In [11]:
train_images, test_images = train_test_split(image_list, test_size=0.2, random_state=42)
print(len(image_list))
print(len(train_images))
print(len(test_images))

18720
14976
3744


In [12]:
df_path = df.groupby('path')
# path_list = list(set(df.path))
train_info = {} 
test_info = {}

for path in train_images :
    temp_df = df_path.get_group(path)
#     p = path.split('/original/')[-1]
    label = temp_df[['label', 'xmin', 'ymin', 'xmax', 'ymax', 'label_id']].values
    train_info[path] = label
    
for path in test_images :
    temp_df = df_path.get_group(path)
#     p = path.split('/original/')[-1]
    label = temp_df[['label', 'xmin', 'ymin', 'xmax', 'ymax', 'label_id']].values
    test_info[path] = label    

In [13]:
np.save('../data/train.npy', train_info)
np.save('../data/test.npy', test_info)

In [15]:
#list(test_info.keys())
#test_info.keys()
test_info['/home/Dataset/Papsmear/original/SS2/06(201029)/20201029_100215.jpg'][:,0]
test_info['/home/Dataset/Papsmear/original/SS2/06(201029)/20201029_100215.jpg']

array([['ASCUS', 1875, 1135, 2007, 1237, 0],
       ['ASCUS', 2132, 967, 2235, 1079, 0],
       ['ASCUS', 2297, 779, 2374, 877, 0],
       ['ASCUS', 2419, 672, 2528, 774, 0]], dtype=object)

In [13]:
transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1280,1280, True,1),
#    A.RandomSizedBBoxSafeCrop(384, 384),
    A.Resize(640, 640, p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomRain(p=0.1),
    A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.99, label_fields=['labels']))

In [14]:
class PapsDataset(Dataset) :
    def __init__(self,image_info,transforms=None):
        self.images_info = image_info
        self.transforms = transforms
        self.image_list = list(image_info.keys())
        
    def __len__(self) -> int:
        return len(self.image_list)        
    
    def __getitem__(self,index):

        path = self.image_list[index]
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = switch_image(image)
        image /= 127.5
        image -= 1
              
        # DETR takes in data in coco format 
        value = self.images_info[path]
        boxes = value[:,1:5] 
        labels = value[:, 5]

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image  = sample['image']
            boxes  = sample['bboxes']
            labels = sample['labels']

        _,h,w = image.shape
        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
        
        ## detr uses center_x,center_y,width,height !!
        if len(boxes)>0:
            boxes = np.array(boxes)
            boxes[:,2:] = (boxes[:,2:] - boxes[:,:2])/2
            boxes[:,:2] += boxes[:,2:]
        else:
            boxes = np.zeros((0,4))
    
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
#        target['image_path'] = torch.tensor([path])
        
        return image, target, path    

In [15]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [29]:
batch_size = 32
train_ds = PapsDataset(train_info,transforms)

train_data_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     if 'down_fc.' not in state :
#         param.requires_grad = False
#     else :
#         param.requires_grad = True

In [53]:
type(pretrained_retinanet.parameters())
type(pretrained_retinanet.state_dict())

collections.OrderedDict

In [ ]:
# pretrained_retinanet.state_dict().keys()

In [69]:
%time
PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
pretrained_retinanet = model.resnet50(num_classes=80,)
pretrained_retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))

retinanet = model.resnet50(num_classes=5,)
for param, state in zip(pretrained_retinanet.parameters(), pretrained_retinanet.state_dict()) :
    #print(state)
    if 'classificationModel' not in state :
        retinanet.state_dict()[state] = param
    else :
        print(state)
        
for param, state in zip(pretrained_retinanet.fpn.parameters(), pretrained_retinanet.fpn.state_dict()) :
    #print(state)
    retinanet.fpn.state_dict()[state] = param
    
for param, state in zip(pretrained_retinanet.regressionModel.parameters(), pretrained_retinanet.regressionModel.state_dict()) :
    #print(state)
    retinanet.regressionModel.state_dict()[state] = param    

            
    

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.68 µs


In [70]:
device = torch.device('cuda')
# device = torch.device('cpu')
retinanet.to(device)
retinanet = torch.nn.DataParallel(retinanet).to(device)
# retinanet.module.freeze_bn()

In [30]:
#len(train_data_loader)

In [72]:
#for i, data in enumerate(tqdm(train_data_loader)) :
for step, data in enumerate(tqdm(train_data_loader, total=len(train_data_loader),leave=False)) :
    images, targets, paths = data
    batch_size = len(images)
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    output = retinanet(images)
    loss_dict = criterion(output, targets)    


  0%|          | 0/468 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory